In [1]:
import pdb

import os
from os.path import join, splitext, basename, dirname, abspath
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn as sk
%matplotlib inline

In [3]:
from keras import initializations

Using Theano backend.


In [4]:
import trading as trd
from trading import State, train_model

In [5]:
# relaod module after modifying it
import importlib
importlib.reload(trd)
importlib.reload(trd.stock_history)
importlib.reload(trd.portfolio)
importlib.reload(trd.benchmarks)
importlib.reload(trd.rl)
importlib.reload(trd.nn)
importlib.reload(trd)

<module 'trading' from 'C:\\Users\\hamza\\Documents\\school\\cs_229\\cs229_final_project\\src\\trading\\__init__.py'>

In [6]:
MODEL_DATA_LOC = '../data/model'

## The Problem

In [7]:
# transaction cost to buy/sell a stock
trans_cost = 0.001
# starting cash
cash = 1e6
# starting portfolio allocation (%lo, %hi)
starting_weights = (0.5, 0.5)
# number of days input
d = 7
# pentalty on volatility for reward method
λ = 0.5
#reward = trd.create_penalized_returns_reward(λ)
#reward_name = 'pen_reward_l={}'.format(λ)
reward = trd.sharpe_ratio_reward
reward_name = 'sharpe_reward'

## The Data

In [8]:
train_data, test_data = trd.get_stock_pairs(m)

# size of training set
m_train = len(train_data)
m_test = len(test_data)
# total number of portfolios
M = m_train + n

portfolio_states = [State(p, cash=cash, target_weights=starting_weights, trans_cost=trans_cost, d=d) for p in train_data]

## A Possible Solution

In [9]:
(model, train_record) = train_model(portfolio_states, trd.actions, 
                                    D=6, gamma=0.99, eps=0.15, H=100,
                                    non_lin='relu', reward=reward, tau=0.001,
                                    debug=True, debug_every=5000)

iter:        0	loss:  6.58368e+07     
iter:     5000	loss:  1.13338e+09     
iter:    10000	loss:  3.21967e+10     
iter:    15000	loss:  1.33903e+09     


In [10]:
#pdb.pm()

In [11]:
raise Exception

Exception: 

## Testing

In [ ]:
test_states = [State(p, cash=cash, target_weights=starting_weights, trans_cost=trans_cost) for p in test_data]
available_test_states = test_states[:]

In [ ]:
while True:
    if available_test_states == []:
        break
    
    states = np.array([st.state for st in available_test_states])
 
    qvalues = model.predict(states)
    chosen_actions = np.argmax(qvalues, axis=1)
    
    for (st, a) in zip(available_test_states, trd.actions[chosen_actions]):
        # execute the action
        st.execute_trade(a)

        try:
            st.step()
        except StopIteration:
            available_test_states.remove(st)

In [ ]:
MODEL_DIR = '{:d}h_{:s}_{:d}eps_{:d}init'.format(H, non_lin, int(ϵ*100), int(cash))
MODEL_DIR = join(MODEL_DATA_LOC, MODEL_DIR)
try:
    os.mkdir(MODEL_DIR)
except Exception:
    pass

In [ ]:
model.save(join(MODEL_DIR, 'model.h5'))

In [ ]:
for st in test_states:
    file_name = '{:s}_{:s}.csv'.format(st.lo.symbol, st.hi.symbol)
    st.portfolio.to_csv(join(MODEL_DIR, file_name))

In [ ]:
def cust_plt(ax, data, label='', **kwargs):
    ax.plot_date(data.index, data, ls='solid', marker='', label=label, **kwargs)

In [ ]:
# number of test examples
N = M - m

matplotlib.rc('font', size=16)
f = plt.figure(figsize=(25, 80))

for i, st in enumerate(test_states):
    lo_name = st.lo.symbol
    hi_name = st.hi.symbol
    lo = st.portfolio.cost_lo
    hi = st.portfolio.cost_hi
    q = st.portfolio.total
    
    ax1 = plt.subplot(N, 2, 2*i + 1)
    cust_plt(ax1, lo, lo_name)
    cust_plt(ax1, hi, hi_name)
    ax1.legend(bbox_to_anchor=(0, 1), loc='upper left', ncol=1)
    plt.xticks(rotation=40)
    
    do_nothing = trd.do_nothing_benchmark(lo, hi, 
                                          trans_cost=trans_cost, 
                                          initial_value=cash)
    rebal = trd.rebalance_benchmark(lo, hi, rebalance_period=30, 
                                    initial_value=cash, 
                                    trans_cost=trans_cost)

    ax2 = plt.subplot(N, 2, 2*i + 2)
    lbl_str = '{:s} ({:.5f})'
    cust_plt(ax2, do_nothing.total, lbl_str.format('do nothing', reward(do_nothing)))
    cust_plt(ax2, rebal.total, lbl_str.format('rebalance 30', reward(rebal)))
    cust_plt(ax2, st.portfolio.total, lbl_str.format('Q Learning', reward(st.portfolio)))
    ax2.axhline(y=cash, color='black')
    ax2.legend(bbox_to_anchor=(0, 1), loc='upper left', ncol=1)
    ax2.set_ylim(bottom=0)
    plt.xticks(rotation=40)

In [ ]:
f.savefig('nn_cmp.png')